# Bosonic Double-well

In [ ]:
using KadanoffBaym

using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## Model

### Non-Hermitian Hamiltonian:

$$
    \hat{H} = \left(\omega_1 -i\frac{\lambda}{2}\right) a^{\dagger}_1 a^\phantom{\dagger}_1 -i\frac{\gamma}{2} a^\phantom{\dagger}_1 a^{\dagger}_1  + \omega_2 a^{\dagger}_2 a^\phantom{\dagger}_2 + J \left(a^{\dagger}_1 a^\phantom{\dagger}_2 + a^{\dagger}_2 a^\phantom{\dagger}_1\right)
$$

### Master equation:

$$
    \partial_{t} \hat{\rho}=-i\left[\hat{H} \hat{\rho}-\hat{\rho} \hat{H}^{
    \dagger}\right]+\lambda a^\phantom{\dagger}_1 \hat{\rho}  a^{\dagger}_1 +\gamma  a^{\dagger}_1 \hat{\rho} a^\phantom{\dagger}_1
$$

### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 + i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    - i\gamma
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & G^{\tilde{T}}_{12} \\
    0 & 0
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 - i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')  
    + i\lambda
    \begin{pmatrix}
    G^T_{11} & G^T_{12} \\
    0 & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 + i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix} 
    - i\gamma
    \begin{pmatrix}
    G^{{T}}_{11} & 0 \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(t, t')     \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 - i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix}  
    + i\lambda
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & 0 \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_T + i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right] 
    - i\gamma
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^\tilde{T}_{12} \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(T, 0)       \\
    0 &= \begin{pmatrix}
    i \partial_T - i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
    + i\lambda
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^T_{12} \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(T, 0)   
\end{split}\end{align}

## Solving

In [ ]:
# time parameters
T = 5.0

# Non-Hermitian Hamiltonian and jump operator
ω₁ = 0.1 * 5.0
ω₂ = 0.0
J = 0.5*2*pi*1.0/2 
U = 0*1.0
λ = 1.0/10.
γ = 0*1.0
h = ComplexF64[ω₁ - 0.5im * (λ + γ) J; J ω₂];

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the end!
gfL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Lesser)
gfG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Greater)

# Note to Tim: Accessing a gf with 2 indices gives you the gf at those time points
# Lesser[1,1] <=> Lesser[:,:,1,1]

# initial condition
N_0 = 5

gfL[1,1,1,1] = -1.0im * N_0
gfG[1,1,1,1] = -1.0im * (N_0 + 1)
gfG[2,2,1,1] = -1.0im

u0 = [gfL, gfG];

function f_vert(u, times, t, t′)
    u1, u2 = u[1], u[2]
    rhs1 = -1.0im * (h * u1[t,t′] + [[1.0im * γ, 0] [0, 0]] * u1[t,t′])
    rhs2 = -1.0im * (h * u2[t,t′] - [[1.0im * λ, 0] [0, 0]] * u2[t,t′])
  return [rhs1, rhs2]
end

function f_diag(u, times, t)
    println("t: $(times[t])")
    u1, u2 = u[1], u[2]
    rhs1 = -1.0im * (h * u1[t,t] - u1[t,t] * adjoint(h)
        + 1.0im * γ * [[u1[1,1,t,t] + u2[1,1,t,t], u2[2,1,t,t]] [u2[1,2,t,t], 0]])
    rhs2 = -1.0im * (adjoint(h) * u2[t,t] - u2[t,t] * h
        - 1.0im * λ * [[u1[1,1,t,t] + u2[1,1,t,t], 0.5 * (u1[2,1,t,t] + u2[2,1,t,t])] [0.5 * (u1[1,2,t,t] + u2[1,2,t,t]), 0]])
  return [rhs1, rhs2]
end

# Basically this will mutate Lesser and Greater in place
sol, _ = kbsolve(f_vert, f_diag, u0, 0.0, T; init_dt=1e-3, atol=1e-5, rtol=1e-3);

In [ ]:
# [(t, tp) for t in sol.t, tp in sol.t]

In [ ]:
# println(map(x -> round(x[2] - x[1], sigdigits=4), zip(sol.t[1:end-1], sol.t[2:end])))

In [ ]:
length(sol.t)

In [ ]:
# n = 128;
n = length(sol.t) - 1;

In [ ]:
# If you want to compare with qutip
# maybe it's best to interpolate the results into an equidistant time-domain

new_time = range(first(sol.t), stop=last(sol.t), length=n + 1)

using Interpolations

gfL_11_itp = interpolate((sol.t, sol.t), gfL.data[1,1,:,:], Gridded(Linear()))
gfL_22_itp = interpolate((sol.t, sol.t), gfL.data[2,2,:,:], Gridded(Linear()))
gfG_11_itp = interpolate((sol.t, sol.t), gfG.data[1,1,:,:], Gridded(Linear()))
gfG_22_itp = interpolate((sol.t, sol.t), gfG.data[2,2,:,:], Gridded(Linear()))

gfL_11_equi = [gfL_11_itp(t1, t2) for t1=new_time, t2=new_time];
gfL_22_equi = [gfL_22_itp(t1, t2) for t1=new_time, t2=new_time];
gfG_11_equi = [gfG_11_itp(t1, t2) for t1=new_time, t2=new_time];
gfG_22_equi = [gfG_22_itp(t1, t2) for t1=new_time, t2=new_time];

In [ ]:
stop = Int(round(n/2)) + 1 

times = sol.t # range(first(sol.t), stop=last(sol.t), length=n + 1) |> collect

# defining the tau times
times_tau = 2 .* vcat([-times[stop - (k - 1)] for k in 1:stop - 1], times[1:stop]);

In [ ]:
stop

## QuTiP benchmark

In [ ]:
print(qt.Options().method)

In [ ]:
# maximal number of photons
# Fock-space dimension will be n_max + 1
n_max = 10

# initial state
psi0_list = [qt.basis(n_max + 1, N_0), qt.basis(n_max + 1, 0)]
psi0 = qt.tensor(psi0_list)

# define annihilation operators
a_1_list = [qt.destroy(n_max + 1), qt.qeye(n_max + 1)]
a_1  = qt.tensor(a_1_list)    
a_2_list = [qt.qeye(n_max + 1), qt.destroy(n_max + 1)]
a_2  = qt.tensor(a_2_list)    

# Hamiltonian
H  = J * a_1.dag() * a_2
H += H.dag()
H += ω₁ * a_1.dag() * a_1 + ω₂ * a_2.dag() * a_2
H += 0.5 * U *(a_1.dag() * a_1 * a_1.dag() * a_1 + a_2.dag() * a_2 * a_2.dag() * a_2)

observables = [a_1.dag()*a_1, a_2.dag()*a_2, a_1*a_1.dag(), a_2*a_2.dag()];

In [ ]:
j_ops = [sqrt(λ) * a_1, sqrt(γ) * a_1.dag()] # qutip only needs the jump operator

# quickly solve once for observables
me = qt.mesolve(H, psi0, times, j_ops, observables)

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, psi0, times, j_ops); # t_sols.states returns density matrices

#### Two times

Calculate $\langle a^\dagger_1(t') a^\phantom{\dagger}_1(t)\rangle$, where $t' = t + \tau$.

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, a_1 * t_sols.states[k], vcat(times[k:end] .- times[k], 
            times[end] .+ (1:(k - 1) |> collect) .* (times[end] - times[end-1])), j_ops).states
end

a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        a_1_dag_a_1[k, l] = (a_1.dag() * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
tau_t_sols = Dict()
for k in 1:length(t_sols.states)
    tau_t_sols[k] = qt.mesolve(H, a_2 * t_sols.states[k], times, j_ops).states
end

a_2_dag_a_2 = zeros(ComplexF64, length(t_sols.states), length(t_sols.states))
for k in 1:length(t_sols.states)
    for l in 1:length(t_sols.states)
        a_2_dag_a_2[k, l] = (a_2.dag() * tau_t_sols[k][l]).tr()    
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_a_1_dag_a_1 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([a_1_dag_a_1[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_1_dag_a_1[k, ind] = y  
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_a_2_dag_a_2 = zeros(ComplexF64, length(t_sols.states), 2*length(t_sols.states) - 1)
for (k, x) in enumerate([a_2_dag_a_2[k, :] for k in 1:length(t_sols.states)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_2_dag_a_2[k, ind] = y  
    end
end

In [ ]:
figure(figsize=(6, 2))
subplot(121)
imshow(real(a_1_dag_a_1), cmap="plasma")

subplot(122)
imshow(real(unskewed_a_1_dag_a_1), cmap="plasma")
colorbar()

tight_layout()

## Plotting

In [ ]:
imshow(-imag(gfL.data[1, 1, :, :]))

In [ ]:
# time diagonal
GL_11 = [-imag(gfL_11_equi[k, k]) for k in 1:n+1]
GL_22 = [-imag(gfL_22_equi[k, k]) for k in 1:n+1];

GG_11 = [-imag(gfG_11_equi[k, k]) for k in 1:n+1]
GG_22 = [-imag(gfG_22_equi[k, k]) for k in 1:n+1];

In [ ]:
# building the tau Green functions
GL_11_tau = [gfL.data[1, 1, stop - (k - 1), stop + (k - 1)] for k in 1:stop]
GL_11_tau_reversed = [GL_11_tau[stop - (k - 1)] for k in 1:stop];
GL_11_tau_full = vcat(-conj(GL_11_tau_reversed[1:end-1]), GL_11_tau);

GL_22_tau = [gfL.data[2, 2, stop - (k - 1), stop + (k - 1)] for k in 1:stop]
GL_22_tau_reversed = [GL_22_tau[stop - (k - 1)] for k in 1:stop];
GL_22_tau_full = vcat(-conj(GL_22_tau_reversed[1:end-1]), GL_22_tau);

In [ ]:
a_1_dag_a_1_tau = [unskewed_a_1_dag_a_1[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_reversed = [a_1_dag_a_1_tau[stop - (k - 1)] for k in 1:stop]
a_1_dag_a_1_tau_full = vcat(conj(a_1_dag_a_1_tau_reversed[1:end-1]), a_1_dag_a_1_tau);

a_2_dag_a_2_tau = [unskewed_a_2_dag_a_2[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
a_2_dag_a_2_tau_reversed = [a_2_dag_a_2_tau[stop - (k - 1)] for k in 1:stop]
a_2_dag_a_2_tau_full = vcat(conj(a_2_dag_a_2_tau_reversed[1:end-1]), a_2_dag_a_2_tau);

In [ ]:
figure(figsize=(12, 6))
subplot(221)
plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="r")
plot(sol.t, [-imag(gfL.data[2, 2, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="r")
plot(times, me.expect[1], c="k", ls="--", lw=3.0, alpha=0.5)
plot(times, [real(unskewed_a_2_dag_a_2[k, k]) for k in 1:n+1], c="k", ls="--", lw=3.0, alpha=0.5)
# plot(times, me.expect[2], c="k", ls="--")
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(222)
idx = 50
plot(sol.t[idx:end - 1], [-imag(gfL.data[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", ms=1, c="r")
# plot(sol.t[idx:end - 1], [real(unskewed_a_1_dag_a_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], c="k", ls="--", lw=3.0, alpha=0.5)
plot(sol.t[idx:end - 1], [real(unskewed_a_1_dag_a_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", c="k", lw=3.0, alpha=0.5, ms=1)

# plot(times_tau, -imag(GL_11_tau_full), "o", ms=2)
# plot(times_tau, a_1_dag_a_1_tau_full, c="k", "s", lw=3.0, alpha=0.5, ms=2)
# plot(times_tau, -imag(GL_22_tau_full), "o", ms=2)
# plot(times_tau, a_2_dag_a_2_tau_full, c="k", "s", lw=3.0, alpha=0.5, ms=2)

# xlim(-T, T)
xlabel("\$\\tau\$")
# ylim(-5.0, 5.0)

subplot(223)
plot(sol.t, [-imag(gfG.data[1, 1, k, k]) for k in 1:length(sol.t)] .- 1, marker="", ms=3.0, ls="-", c="r")
plot(sol.t, [-imag(gfG.data[2, 2, k, k]) for k in 1:length(sol.t)] .- 1, marker="", ms=3.0, ls="-", c="r")
plot(times, me.expect[3] .- 1, c="k", ls="--", lw=3.0, alpha=0.5)
plot(times, me.expect[4] .- 1, c="k", ls="--", lw=3.0, alpha=0.5)
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

tight_layout()
# savefig("test.pdf")

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)] .- me.expect[1], marker="", ms=3.0, ls="-", c="r")
# plot(times, GL_11 - me.expect[1], c="k", ls="--")
# plot(times, GL_22 - me.expect[2], c="k", ls="--")
# plot(times, GG_11 - me.expect[3], c="k", ls=":")
# plot(times, GG_22 - me.expect[4], c="k", ls=":")
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")

subplot(122)
idx = 20
plot(sol.t[idx:end - 1], [-imag(gfL.data[1, 1, idx, k + idx - 1]) for k in 1:length(sol.t) - idx] 
    .- [real(unskewed_a_1_dag_a_1[idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", ms=1, c="r")
# plot(times_tau, -imag(GL_11_tau_full) - a_1_dag_a_1_tau_full, c="k", ls="--")
# plot(times_tau, -imag(GL_22_tau_full) - a_2_dag_a_2_tau_full, c="k", ls="--")
xlim(-T, T)
xlabel("\$\\tau\$")
# ylim(0, 1.0)

tight_layout()

## Testing

In [ ]:
idx_1 = 20
plot(sol.t, [-imag(gfL.data[1, 1, idx_1, k]) for k in 1:n+1], "o", ms=3.0, c="r")
plot(sol.t, [a_1_dag_a_1[idx_1, k] for k in 1:n+1], c="k", "s", lw=3.0, alpha=0.5, ms=2)

In [ ]:
[wigner_transform(gfL.data[1, 1, :, :]; fourier=false)[1] for k in length(sol.t)]

In [ ]:
figure(figsize=(12, 3))
subplot(121)
xlim(0, T)
plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)], marker="o", ms=3.0, ls="")
plot(sol.t, [-imag(gfG.data[1, 1, k, k]) for k in 1:length(sol.t)] .- 1, marker="x", ms=4.0, ls="")
tight_layout()

In [ ]:
figure(figsize=(12, 3))
subplot(121)
xlim(0, T/2)
plot(sol.t, [-imag(gfL.data[1, 1, k, k]) for k in 1:length(sol.t)] .- ([-imag(gfG.data[1, 1, k, k]) for k in 1:length(sol.t)] .- 1), marker="o", ms=3.0, ls="")
tight_layout()